In [15]:
import requests

cookies = {
    'JSESSIONID': 'B7265EAA789B2654DBC7005531EFB13A',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9,tr;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    # 'Cookie': 'JSESSIONID=B7265EAA789B2654DBC7005531EFB13A',
    'Origin': 'http://tools.nlp.itu.edu.tr',
    'Referer': 'http://tools.nlp.itu.edu.tr/IsTurkish',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
}


In [16]:
from bs4 import BeautifulSoup


In [17]:
data = {
    'input': 'şıpsevdiler\r\nşıpıdıkçı\r\nşıpır\r\nşıpırtı\r\nşıpşıp\r\nşıra\r\nşırak\r\nşırakkadak\r\nşırfıntı\r\nşırla\r\nşırlağan\r\nşırıl\r\nşırıltı\r\nşırınga\r\nşıvgın',
    'output': '',
}

response = requests.post('http://tools.nlp.itu.edu.tr/IsTurkish', cookies=cookies, headers=headers, data=data, verify=False)
response.status_code

200

In [6]:

soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
lines = soup.find('textarea', {'name': 'output'}).text.split('\n')
lines

['şıpsevdiler: true\r',
 'şıpıdıkçı: true\r',
 'şıpır: true\r',
 'şıpırtı: true\r',
 'şıpşıp: true\r',
 'şıra: true\r',
 'şırak: true\r',
 'şırakkadak: true\r',
 'şırfıntı: false\r',
 'şırla: true\r',
 'şırlağan: true\r',
 'şırıl: true\r',
 'şırıltı: true\r',
 'şırınga: true\r',
 'şıvgın: true\r',
 '']

In [8]:
lines = soup.find('textarea', {'name': 'output'}).text.split('\n')

is_turkish_map = {}
for line in lines:
    if line.strip() == '' or len(line.split()) < 2:
        continue
    parts = line.split(':')
    word = parts[0].strip()
    
    is_turkish_map[word] = parts[1].startswith(' t')

is_turkish_map

{'şıpsevdiler': True,
 'şıpıdıkçı': True,
 'şıpır': True,
 'şıpırtı': True,
 'şıpşıp': True,
 'şıra': True,
 'şırak': True,
 'şırakkadak': True,
 'şırfıntı': False,
 'şırla': True,
 'şırlağan': True,
 'şırıl': True,
 'şırıltı': True,
 'şırınga': True,
 'şıvgın': True}

In [9]:
def get_is_turkish(text):
  is_turkish_map = {}

  data = {
      'input': text,
      'output': '',
  }

  response = requests.post('http://tools.nlp.itu.edu.tr/IsTurkish', cookies=cookies, headers=headers, data=data, verify=False)

  soup = BeautifulSoup(response.text, 'html.parser')

  lines = soup.find('textarea', {'name': 'output'}).text.split('\n')

  for line in lines:
      if line.strip() == '' or len(line.split()) < 2:
          continue
      parts = line.split(':')
      word = parts[0].strip()

      is_turkish_map[word] = parts[1].startswith(' t')

  return is_turkish_map

In [10]:
kokler_set = set()

with open("veri/after_itu_kokler.txt", "r", encoding="utf-8") as f:
    for satir in f.readlines():
        kokler_set.add(satir.strip().split(' ')[0])

len(kokler_set)

23653

In [21]:
chunk_size = 1361
chunks = [list(kokler_set)[i:i + chunk_size] for i in range(0, len(kokler_set), chunk_size)]

text = " ".join(chunks[10])

get_is_turkish(text)

{'levüloz': True,
 'sifonlular': True,
 'mübadele': True,
 'yabansı': True,
 'monokl': True,
 'düze': True,
 'müdavi': False,
 'tagaddi': True,
 'magmatik': True,
 'sığırgöz': False,
 'tesis': True,
 'kıtaat': True,
 'bendegan': False,
 'hinoğlu': False,
 'enterne': True,
 'tahtelbahir': True,
 'disimilasyon': True,
 'maharet': True,
 'buzuki': True,
 'manolyagiller': True,
 'reçete': True,
 'norm': True,
 'falanj': True,
 'elektromanyetik': True,
 'payitaht': True,
 'ilkyardım': True,
 'okullu': True,
 'tül': True,
 'belirlenmezcilik': True,
 'taraz': True,
 'şömine': True,
 'fototaksi': True,
 'likorinoz': True,
 'eriyik': True,
 'ürkek': True,
 'batkın': True,
 'köşebaşı': True,
 'oniks': True,
 'safsata': True,
 'viran': True,
 'mahmur': True,
 'pardösü': True,
 'hidrojeoloji': True,
 'nihai': True,
 'yüzergezer': True,
 'tababet': True,
 'maskarata': True,
 'aklıevvel': True,
 'insanoğlu': True,
 'hatm': False,
 'saki': True,
 'bilişim': True,
 'mızmız': True,
 'miyav': True,
 'me

In [22]:
is_turkish_map = {}

for i, chunk in enumerate(chunks):
    text = " ".join(chunk)
    is_turkish_map.update(get_is_turkish(text))
    print(f"{i}/{len(chunks)}")

len(is_turkish_map)

0/18
1/18
2/18
3/18
4/18
5/18
6/18
7/18
8/18
9/18
10/18
11/18
12/18
13/18
14/18
15/18
16/18
17/18


23645

In [23]:
# turkish words

turkish_words = {k: v for k, v in is_turkish_map.items() if v}
len(turkish_words)

21023

In [27]:
import pandas as pd

df = pd.DataFrame(is_turkish_map.items(), columns=['word', 'is_turkish'])
df.to_csv("veri/itu_turkish_words.csv", index=False, encoding="utf-8")
df

,word,is_turkish
0,muntazır,False
1,banla,True
2,miksefe,False
3,fon,True
4,necaset,True
...,...,...
23640,modistra,True
23641,tezle,True
23642,yayan,True
23643,avrupai,True


In [26]:
with open("veri/itu_turkish_kokler.txt", "w", encoding="utf-8") as f:
    kok_sirali_liste = sorted(list(turkish_words.keys()))
    for kok in kok_sirali_liste:
        print(kok, file=f, flush=True)